In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
import numpy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import SGDRegressor
import pickle
import cv2
import math


In [3]:
OLS_model = pickle.load(open('OLS.pkl', 'rb'))
SGD_model = pickle.load(open('SGD.pkl', 'rb'))
MeClf_model = pickle.load(open('Meclassifier.pkl', 'rb'))



In [4]:
img_dim = 64

In [62]:
def faceDetect(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=3,
        minSize=(img_dim, img_dim)
    )
    
    if(len(faces) > 1):
        cur_max = 0
        face = 0

        for (x, y, w, h) in faces:
            length = math.sqrt((x - (x+w))**2 + (y - (y+h))**2 )
            
            if length > cur_max:
                cur_max = length
                face = [x,y,w,h]

        faces = np.array([face])

    for (x, y, w, h) in faces:
    
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # Draw a rectangle around the detected area (face)

        roi_color = image[y+2:y+2 + h-3, x+2:x+2 + w-3] # The pixel area containing the face

        return [image,roi_color]
    
    return [0,[0]]

In [98]:
cv2.destroyAllWindows()

cam = cv2.VideoCapture(0)

while True:
    face_detected = True

    check, frame = cam.read()

    frame = cv2.flip(frame,1)
    
    frame,frame_cropped = faceDetect(frame)

    frame = cv2.copyMakeBorder(frame,1,150,1,1,cv2.BORDER_CONSTANT,value=0)

    cv2.putText(frame,"Who:",(20,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
    cv2.putText(frame,"Age:",(370,530),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)


    cv2.imshow('video', frame)


    if(len(frame_cropped)>=2):
        img_resize_clf = cv2.resize(frame_cropped, (64,64))
        img_resize_reg = cv2.resize(frame_cropped, (32,32))

    else:
        face_detected = False

    if(face_detected):
        cur_image_clf = img_resize_clf.flatten()
        cur_image_ref = img_resize_reg.flatten()

        me_pred = MeClf_model.predict([cur_image_clf])
        ols_pred = OLS_model.predict([cur_image_ref])
        sgd_pred = SGD_model.predict([cur_image_ref])

        if(me_pred == 0):
            cv2.putText(frame,"Unknown",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            cv2.putText(frame,"OLS - " + str(int(ols_pred)),(370,580),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
            cv2.putText(frame,"SGD - " + str(int(sgd_pred)),(370,620),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)

            cv2.imshow('video', frame)
        else:
            cv2.putText(frame,"Sameed",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
            cv2.putText(frame,"OLS - " + str(int(ols_pred)),(370,580),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
            cv2.putText(frame,"SGD - " + str(int(sgd_pred)),(370,620),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1)
            cv2.imshow('video', frame)

    else:
        cv2.putText(frame,"Face Not Detected",(20,580),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1)
        cv2.imshow('video', frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 